In [49]:
import folium
import openrouteservice as ors
import math
import pandas as pd

In [50]:
deliveries_data = pd.read_csv(
    'idai_health_sites (9).csv',
    index_col="ID",
    parse_dates=["Open_From", "Open_To"]
)

# Simple single route optimization
coords = [
    [13.17644, 80.09819],
    [13.18882, 80.10655],
    [13.16864, 80.09842],
    [13.15337, 80.07978],
    [13.13483, 80.10205],
    [13.12671, 80.14414],
    [13.12220, 80.10223],
    [13.14892, 80.05922],
]
# visualize the points on a map
m = folium.Map(location=list([13.16401, 80.11519]), tiles="cartodbpositron", zoom_start=12.5)
for coord in coords:
    folium.Marker(location=list(coord)).add_to(m)
m

In [51]:
client = ors.Client(key='5b3ce3597851110001cf6248851067b8fe2c4cee869de271e0249a32')

# 12.96053, 77.50021 ban
# 80.060034, 13.135152
coords = [
    #[13.17644, 80.09819],
    #[13.18882, 80.10655],
    [13.16864, 80.09842],
    [13.15337, 80.07978],
    [13.13483, 80.10205],
    [13.12671, 80.14414],
    [13.12220, 80.10223],
    [13.14892, 80.05922],
]
vehicle_start = [13.135152, 80.060034]
m = folium.Map(location=list([13.16401, 80.11519]), tiles="cartodbpositron", zoom_start=12.5)
for coord in coords:
    folium.Marker(location=list(coord)).add_to(m)
    
folium.Marker(location=list(vehicle_start), icon=folium.Icon(color="red")).add_to(m)


vehicle_end0 = [13.17644, 80.09819]
vehicle_end1 = [13.18882, 80.10655]
folium.Marker(location=list(vehicle_end0), icon=folium.Icon(color="green")).add_to(m)
folium.Marker(location=list(vehicle_end1), icon=folium.Icon(color="pink")).add_to(m)
m






In [52]:
#vehicles = ors.optimization.Vehicle(id:1,start:[80.10223, 13.12220],end:[80.10223, 13.12220],capacity:[4])

#depot = ( 13.12220, 80.10223)
vehicles = list()
#end_points=[[13.17644, 80.09819],[13.18882, 80.10655],[13.12220, 80.10223]]
for idx in range(3):
    vehicles.append(
        ors.optimization.Vehicle(
            id=idx,
            start=list(reversed(vehicle_start)),
            #end=list(reversed(end_points[idx])),
            capacity=[300],
            time_window=[1553241600, 1553284800]  # Fri 8-20:00, expressed in POSIX timestamp
        )
    )
    print(idx)


#[{"id":1,"profile":"driving-car","start":[80.10223, 13.12220],"end":[80.05922, 13.14892],"capacity":[4],"skills":[1,14],
 # "time_window":[28800,43200]},
 #{"id":2,"profile":"driving-car","start":[80.05922, 13.14892],"end":[80.05922, 13.14892],"capacity":[4],"skills":[2,14],
  #"time_window":[28800,43200]}]

0
1
2


In [53]:
#jobs = ors.optimization.Job([{"id":1,"service":300,"delivery":[1],"location":[80.09819, 13.17644],"skills":[1],"time_windows":[[32400,36000]]},{"id":2,"service":300,"delivery":[1],"location":[80.10655, 13.18882],"skills":[1]},{"id":3,"service":300,"delivery":[1],"location":[80.09842, 13.16864],"skills":[2]},{"id":4,"service":300,"delivery":[1],"location":[80.07978, 13.15337],"skills":[2]},{"id":5,"service":300,"delivery":[1],"location":[80.10205,13.13483],"skills":[14]},{"id":6,"service":300,"delivery":[1],"location":[80.14414, 13.12671],"skills":[14]}]) 
#timestamp(#geometry = client.directions(coords)['routes'][0]['geometry']
    
deliveries = list()
for delivery in deliveries_data.itertuples():
        deliveries.append(
            ors.optimization.Job(
                id=delivery.Index,
                location=[delivery.Lon, delivery.Lat],
                service=1200,  # Assume 20 minutes at each site
                amount=[delivery.Needed_Amount]
        )
    )
        print(delivery)
        print(deliveries)


Pandas(Index=1, Lon=80.09819, Lat=13.17644, Open_From='\t2019-03-2208:00:00', Open_To='\t2019-03-2218:00:00', Needed_Amount=24)
Pandas(Index=2, Lon=80.10655, Lat=13.18882, Open_From='\t2019-03-2208:30:00', Open_To='\t2019-03-2220:00:00', Needed_Amount=10)
[<openrouteservice.optimization.Job object at 0x0000028424406EB0>, <openrouteservice.optimization.Job object at 0x000002842436F880>]


In [54]:
optimized = client.optimization(jobs=deliveries, vehicles=vehicles, geometry=True)


In [55]:
line_colors = ['green', 'orange', 'blue', 'yellow']
for route in optimized['routes']:
    folium.PolyLine(locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']], color=line_colors[route['vehicle']]).add_to(m)
m

In [56]:
extract_fields = ['distance', 'amount', 'duration']
data = [{key: route[key] for key in extract_fields} for route in optimized['routes']]

vehicles_df = pd.DataFrame(data)
vehicles_df.index.name = 'vehicle'
vehicles_df

,distance,amount,duration
vehicle,,,
0,13056,[34],1011


In [57]:
# Create a list to display the schedule for all vehicles
stations = list()
for route in optimized['routes']:
    vehicle = list()
    for step in route["steps"]:
        vehicle.append(
            [
                step.get("job", "Depot"),  # Station ID
                step["arrival"],  # Arrival time
                step["arrival"] + step.get("service", 0),  # Departure time

            ]
        )
    stations.append(vehicle)

In [13]:
df_stations_0 = pd.DataFrame(stations[0], columns=["Station ID", "Arrival", "Departure"])
df_stations_0['Arrival'] = pd.to_datetime(df_stations_0['Arrival'], unit='s')
df_stations_0['Departure'] = pd.to_datetime(df_stations_0['Departure'], unit='s')
df_stations_0

,Station ID,Arrival,Departure
0,Depot,2019-03-22 08:00:00,2019-03-22 08:00:00
1,1,2019-03-22 08:13:21,2019-03-22 08:33:21
2,2,2019-03-22 08:36:51,2019-03-22 08:56:51
3,Depot,2019-03-22 08:56:51,2019-03-22 08:56:51
